# YORP Effect on a Single Asteroid

This example shows the effect of the YORP radiation force on a single asteroid rotating near break-up speed, in orbit around a white dwarf. The fragmentation process is defined in a custom heartbeat function and produces a binary asteroid with a few smaller fragments called "shards". Since we are using test particles (where the mass is set to 0), this is a simplified model and does not include binary interactions between the asteroid fragments. 

First we set up the simulation with a solar-mass star and an asteroid with a semi-major axis of 0.005 AU:

In [1]:
import rebound
import reboundx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Initialize the simulation
sim = rebound.Simulation()

sim.units = ('yr', 'AU', 'Msun') 
sim.integrator = "whfast" 
sim.dt = .0005 # simulation timestep in years

# add a solar-mass star
sim.add(m=1)

# add an asteroid at a = 0.005 AU
sim.add(a=0.005, f=0, Omega=0, omega=0, e=0, inc=0, m=0)

sim.move_to_com()

OSError: /home/kaelynclement/rebound/rebound/.venv/lib/python3.10/site-packages/librebound.cpython-310-x86_64-linux-gnu.so: cannot open shared object file: No such file or directory

Now we set all the parameters needed to run a simulation with the YORP effect:

In [ ]:
# unit conversions
au_conv = 1.495978707e11 # meters in 1 AU
msun_conv = 1.9885e30    # kg in 1 solar mass
yr_conv = 31557600.0     # seconds in a year

# effect properties
phi = 1.e17/au_conv/msun_conv*yr_conv*yr_conv # solar radiation constant
lstar = 1.e-3                                 # star luminosity over solar luminosity

# object properties
radius = 100.0/au_conv
density = (2000.0*au_conv*au_conv*au_conv)/msun_conv # density of the object
c_body = 1./10./10.                                  # amount of asymmetry and obliquity in the asteroid
sigma = 1.0e3/msun_conv*au_conv*yr_conv*yr_conv      # tensile strength of the asteroid
rotation_frequency = 0.0081991*yr_conv               # initial rotation frequency
obliquity = np.pi/6
alpha = 2./3.
beta = 1./3.

Next we must add the YORP effect to the simulation and set the YORP parameters for the asteroid on which we want it to act:

In [ ]:
# Load the YORP effect into REBOUND
rebx = reboundx.Extras(sim)
yorp = rebx.load_operator("yorp_effect")

# set the effect properties
yorp.params["yorp_solar_radiation_constant"] = phi
yorp.params["yorp_lstar"] = lstar

# Set parameters for the particle
ps = sim.particles

ps[1].r = radius # remember radius is not inputted as a REBx parameter - it is inputted on the particle in the sim
ps[1].params["yorp_body_density"] = density
ps[1].params["yorp_c_body"] = c_body
ps[1].params["yorp_tensile_strength"] = sigma
ps[1].params["yorp_rotation_frequency"] = rotation_frequency
ps[1].params["yorp_obliquity"] = obliquity
ps[1].params["yorp_alpha"] = alpha
ps[1].params["yorp_beta"] = beta

rebx.add_operator(yorp) # add yorp operator to the simulation
sim.N_active = 1

Before beginning the simulation, we also set up a custom heartbeat function for fragmentation. The inclusion of fragmentation in a heartbeat function rather than in the YORP code is to allow the user freedom to define their own fragmentation model. In this example, fragmentation will produce a binary asteroid with a mass ratio of 1.5 plus three smaller shards. The failure spin rate is defined using Equation 2 in Veras & Scheeres (2020), but the minimum radius for fragmentation is set somewhat arbitrarily.

In [ ]:
min_r = 10.0/au_conv                # minimum radius at which a particle will break apart
reset_rotation_frequency = 0.0      # particle rotation frequency after fragmentation
primary_mass_ratio = 1.5            # mass ratio between the two primary fragments
primary_secondary_mass_ratio = 50.0 # mass ratio between the primary (binary) and the shards
n_shards = 3                        # number of shards to create

# heartbeat function which defines how fragmentation of a particle due to YORP should be handled
def heartbeat(sim_pointer):
    sim = sim_pointer.contents
    ps = sim.particles
    G = sim.G

    for i in range(1,sim.N):
        # parameters needed to calculate failure spin rate for fragmentation
        r = ps[i].r
        density = ps[i].params["yorp_body_density"]
        sigma = ps[i].params["yorp_tensile_strength"]
        rotation_frequency = ps[i].params["yorp_rotation_frequency"]

        # Eq. 2 in Veras and Scheeres (2020). Actually the failure spin rate squared.
        failure_spin_rate = 4*np.pi*G*density/3.0 + 2.0*sigma/(density*r**2)*(2.0/3.0)

        # break apart only if spinning at or above spin rate AND the radius is greater than min fragment size
        if rotation_frequency**2 >= failure_spin_rate and r > min_r:
            print("Asteroid", i, "break-up at t = ", sim.t, "yr. Rotation frequency at break-up:", rotation_frequency/yr_conv, "rad/s.")
            
            # reset the rotation frequency
            ps[i].params["yorp_rotation_frequency"] = reset_rotation_frequency

            # save the original particle's mass and radius values
            m_old = ps[i].m
            r_old = ps[i].r

            # set mass and radius of first primary fragment
            m_primary = m_old/(1.0+1.0/(primary_secondary_mass_ratio))
            r_primary = (1.0+1.0/(primary_secondary_mass_ratio))**(-1.0/3.0)*r_old
            ps[i].m = m_primary/(1.0+1.0/(primary_mass_ratio))
            ps[i].r = (1.0+1.0/(primary_mass_ratio))**(-1.0/3.0)*r_primary

            # create second primary fragment and add to simulation
            fragment = rebound.Particle()
            fragment.m = m_primary - ps[i].m
            fragment.r = primary_mass_ratio**(-1.0/3.0)*ps[i].r
            fragment.x = ps[i].x + 10*ps[i].r; # offset slightly so not overlapping
            fragment.y = ps[i].y
            fragment.z = ps[i].z
            fragment.vx = ps[i].vx
            fragment.vy = ps[i].vy
            fragment.vz = ps[i].vz
            sim.add(fragment)

            # set yorp-specific parameters for the fragment using parameters from the original particle
            ps[i+1].params["yorp_body_density"] = ps[i].params["yorp_body_density"]
            ps[i+1].params["yorp_c_body"] = ps[i].params["yorp_c_body"]
            ps[i+1].params["yorp_tensile_strength"] = ps[i].params["yorp_tensile_strength"]
            ps[i+1].params["yorp_rotation_frequency"] = reset_rotation_frequency
            
            # set mass and radius totals for shards
            m_secondary = m_old - m_primary
            r_secondary = r_primary*primary_secondary_mass_ratio**(-1.0/3.0)

            # add shards to the simulation
            for j in range(0,n_shards):
                shard = rebound.Particle()
                # shards all have the same mass and radius
                shard.m = m_secondary/n_shards
                shard.r = n_shards**(-1.0/3.0)*r_secondary
                shard.x = ps[i].x + 10*(j+2)*ps[i].r; # offset slightly so not overlapping
                shard.y = ps[i].y
                shard.z = ps[i].z
                shard.vx = ps[i].vx
                shard.vy = ps[i].vy
                shard.vz = ps[i].vz
                sim.add(shard)

                N = sim.N
                # set yorp-specific parameters for the shard using parameters from the original particle
                ps[N-1].params["yorp_body_density"] = ps[i].params["yorp_body_density"]
                ps[N-1].params["yorp_c_body"] = ps[i].params["yorp_c_body"]
                ps[N-1].params["yorp_tensile_strength"] = ps[i].params["yorp_tensile_strength"]
                ps[N-1].params["yorp_rotation_frequency"] = reset_rotation_frequency
                
sim.heartbeat = heartbeat

Now we run the simulation over 10 years, just enough time for the asteroid to fragment and for the fragments to spin up a little bit under the YORP effect.

In [ ]:
%%time
tmax = 10 # in yrs
Nout = 100
times = np.linspace(0, tmax, Nout)

for i, time in enumerate(times):
    sim.integrate(time)

Lastly, we print the final rotation frequency and radius for each asteroid. The original asteroid had a radius of 100 m.

In [ ]:
for i in range(1,sim.N):
    final_w = ps[i].params["yorp_rotation_frequency"]/yr_conv
    final_r = ps[i].r*au_conv
    final_obliq = ps[i].params["yorp_obliquity"]
    print("ASTEROID", i, "FINAL ROTATION FREQUENCY:", final_w, "rad/s")
    print("ASTEROID", i, "FINAL RADIUS:", final_r, "m\n")
    print("ASTEROID", i, "FINAL OBLIQUITY:", final_obliq, "m\n")